## Sokoban analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
games = pd.read_csv('games.csv')
levels = pd.read_csv('levels.csv')

In [ ]:
pattern = '|'.join(['kevin', 'mihkel', 'udam', 'elle', 'test'])

games_filtered = games[~games['id_code'].str.contains(pattern, case=False, na=False)]

games_filtered.describe()


In [ ]:
filtered_game_ids = games_filtered['id']
levels_filtered = levels[levels['game_id'].isin(filtered_game_ids)]

levels_filtered.info()

In [ ]:
#Remove broken Safari games
game_ids_in_levels = levels_filtered['game_id'].unique()
games_filtered = games_filtered[games_filtered['id'].isin(game_ids_in_levels)]

games_filtered.info()


## Results overview

### Level attempts vs successes

In [ ]:
# Count total attempts per level

# drop rows where level_id is 15

levels_filtered = levels_filtered[levels_filtered['level_id'] != 15]
total_attempts = levels_filtered['level_id'].value_counts().reset_index()
total_attempts.columns = ['level_id', 'total_attempts']

# Count how many times the level was passed
passed_attempts = levels_filtered[(~levels_filtered['end_time'].isna()) & (levels_filtered['skipped'] != 't')]['level_id'].value_counts().reset_index()
passed_attempts.columns = ['level_id', 'passed_attempts']

# Merge the two dataframes
df = pd.merge(total_attempts, passed_attempts, on='level_id')

# Calculate success percentage
df['success_percentage'] = df['passed_attempts'] / df['total_attempts'] * 100

# Create a bar plot
plt.figure(figsize=(12,8))
p1 = sns.barplot(data=df, x='level_id', y='total_attempts', color='skyblue', label='Total Attempts')
p2 = sns.barplot(data=df, x='level_id', y='passed_attempts', color='olive', label='Successful Attempts')

plt.legend(title='Attempts')
plt.xlabel('Level ID')
plt.ylabel('Number of Attempts')
plt.title('Total vs Successful Attempts per Level')
plt.show()

## Levels ordered by success rate

In [ ]:
df['level_id'] = df['level_id'].astype(str)
df = df.sort_values(by='success_percentage', ascending=False)

# Create a bar plot
plt.figure(figsize=(12,8))
p1 = sns.barplot(data=df, x='level_id', y='success_percentage', color='skyblue', label='Success Percentage')

# Add percentage text on top of each bar
for p in p1.patches:
    percentage = '{:.1f}%'.format(p.get_height())
    p1.annotate(percentage, (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

plt.legend()
plt.xlabel('Level ID')
plt.ylabel('Success Percentage')
plt.title('Success Percentage per Level')
plt.show()

## Levels ordered by median time

In [ ]:
# Select entries with both end_time and start_time
valid_entries = levels_filtered.dropna(subset=['start_time', 'end_time']).copy()
valid_entries = valid_entries[valid_entries['skipped'] != 't']
valid_entries['level_id'] = valid_entries['level_id'].astype(str)

# Convert end_time and start_time to datetime
valid_entries['end_time'] = pd.to_datetime(valid_entries['end_time'])
valid_entries['start_time'] = pd.to_datetime(valid_entries['start_time'])

# Calculate completion time in seconds
valid_entries.loc[:, 'completion_time'] = (valid_entries['end_time'] - valid_entries['start_time']).dt.total_seconds()
valid_entries = valid_entries[valid_entries['completion_time'] < 1300]

median_times = valid_entries.groupby('level_id')['completion_time'].median().sort_values()

# Order your dataframe according to median_times
valid_entries['level_id'] = pd.Categorical(valid_entries['level_id'], categories=median_times.index, ordered=True)

In [ ]:
plt.figure(figsize=(16,8))
sns.boxplot(x='level_id', y='completion_time', data=valid_entries)
plt.xlabel('Level ID')
plt.ylabel('Completion Time (seconds)')
plt.title('Distribution of Completion Time per Level')
plt.xticks(rotation=90)  # This rotates the x-axis labels for better visibility if you have many levels
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(x=median_times.index, y=median_times.values, color='skyblue')
plt.xlabel('Level ID')
plt.ylabel('Median Completion Time (seconds)')
plt.title('Median Completion Time per Level')
plt.xticks(rotation=90)  # This rotates the x-axis labels for better visibility if you have many levels
plt.show()

## Player progression stats

In [ ]:
# Filter the dataframe
completed_levels = levels_filtered[(~levels_filtered['start_time'].isna()) & 
                                   (~levels_filtered['end_time'].isna()) & 
                                   (levels_filtered['skipped'] == 'f')]

# Calculate completed levels per game
completed_levels_per_game = completed_levels.groupby('game_id')['level_id'].count()

average_levels = completed_levels_per_game.mean()
median_levels = completed_levels_per_game.median()

print(f"Average completed levels per player: {average_levels}")
print(f"Median completed levels per player: {median_levels}")
print("\n")

# print number of games with 1 level completed

print(f"Number of games with 1 level completed: {len(completed_levels_per_game[completed_levels_per_game == 1])}")
print(f"Number of games with 2 levels completed: {len(completed_levels_per_game[completed_levels_per_game == 2])}")
print(f"Number of games with 3 levels completed: {len(completed_levels_per_game[completed_levels_per_game == 3])}")
print(f"Number of games with at least 4 levels completed: {len(completed_levels_per_game[(completed_levels_per_game >= 4)])}")

print("\n")
print(f"Most successful players and their number of levels completed:\n {completed_levels_per_game.sort_values(ascending=False).head(5)}");

# List all ids of games with 1 levels completed
games_with_1_level = completed_levels_per_game[completed_levels_per_game == 1].index.values

print(games_with_1_level)







# Takeaways

* The data gives a pretty good idea of level difficulty and how to order them.
* Success percentage for most of the levels is under 50%. Should we add more easy levels?
* Median completion time was around 1 minute for easy ones, 5 minutes for the medium difficulty and 10 for harder ones. However we need to take into account that skipping was allowed. So when we eliminate skipping more of the slower people are going to finish levels and the median time will go up considerably. Also currently many people played multiple times and that also brought median time down.
* Taking that into account, 10-12 total levels is probably enough for the final game (assuming 1-2 game attempts per student)
* 1/3 of people gave up after the intro level - game too hard? Too little incentive to make an effort? Probably the random hard levels in the beginning threw many people off. Would be nice to test on current students with ordered levels and success-based incentives.
